# Summarise in Arabic

We'll now continue the example with a recipe from the Vectara website where we search our employee handbook.

See here for the REST version of this: https://docs.vectara.com/docs/api-recipes

In [2]:
%pip install -q vectara-skunk-client==0.1.8

Note: you may need to restart the kernel to use updated packages.


In [3]:
import logging

logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S %z')

In [4]:
from vectara.core import Factory

client = Factory().build()

IndentationError: expected an indented block after function definition on line 23 (query.py, line 26)

In [ ]:
admin_service = client.admin_service

corpora = admin_service.list_corpora("02-employee-handbook")

if len(corpora) >= 1:
    for corpus in corpora:
        admin_service.delete_corpus(corpus.id)
else:
    logging.info("No existing corpus with the name client-example")

## Create Our Demo Corpus
We now create our new corpus called, "02-employee-handbook" where we'll upload our documents for this test.

In [ ]:
create_corpus_result = admin_service.create_corpus("02-employee-handbook", description="Example Corpus for use from Jupyter")
logging.info(f"New corpus created {create_corpus_result}")
corpus_id = create_corpus_result.corpusId

## Upload Handbook to Corpus
We now upload our PDF to the corpus which will parse and then encode the document, storing the embeddings in our vector database.

In [ ]:
from pathlib import Path

handbook_path = Path("resources/vectara_employee_handbook.pdf")
indexer_service = client.indexer_service
indexer_service.upload(corpus_id, handbook_path)

In [ ]:
query_service = client.query_service
#response = query_service.query("Can I bring any birds to the Vectara office?", corpus_id)
response = query_service.query("How much PTO is offered to employees each year?", corpus_id)

In [ ]:
print(response.responseSet[0].summary[0].text)

In [ ]:
from dataclasses import asdict
from vectara.util import _custom_asdict_factory
import json

print(json.dumps(asdict(response, dict_factory=_custom_asdict_factory),indent=4))